In [1]:
from collections import Counter
import numpy as np

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    index = next((i for i, v in np.ndenumerate(board[column]) if v == 0), None)
    if (index == None):
        print(f'Error, selected column {column} but is full')
        exit(1)
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [4]:
def get_points(board, player):
    points = 0
    
    for r in range(COLUMN_HEIGHT):
        seq = 0
        for c in range(NUM_COLUMNS):
            if (c == 0 or board[c,r] == -player):
                free_slots = 0

            if (board[c,r] == player):
                seq += 1
            else:
                if (board[c,r] == 0):
                    free_slots += 1

                    if (seq + free_slots >= 4):
                        points += seq * seq
                        seq = 0
                else:
                    if (seq + free_slots >= 4):
                        points += seq * seq
                    seq = 0
                    free_slots = 0

    for c in range(NUM_COLUMNS):
        seq = 0
        for r in range(COLUMN_HEIGHT):
            
            if (board[c,r] == player):
                seq += 1
            else:
                if (board[c,r] == 0):
                    free_slots = COLUMN_HEIGHT - 1 - r
                    if (seq + free_slots >= 4):
                        points += seq * seq
                        seq = 0
                else:
                    seq = 0
    return points

In [5]:
def get_best_move(board, player, deep):
    deep -= 1
    best_move_points = -sys.maxsize
    best_move = -1
    for c in valid_moves(board):
        temp = np.copy(board)
        play(temp, c, player)

        if (four_in_a_row(temp, player)):
            return c

        if (deep != 0):
            opponent_best_move = get_best_move(temp, -player, deep - 1)
            if (opponent_best_move == -1):
                pnt = 0
            else:
                play(temp, opponent_best_move, -player)
                if (four_in_a_row(temp, -player)):
                    opponent_points = sys.maxsize
                else:
                    opponent_points = get_points(temp, -player)
                pnt = get_points(temp, player) - opponent_points
            if (pnt > best_move_points):
                best_move_points = pnt
                best_move = c
        else:
            pnt = get_points(temp, player) - get_points(temp, -player)
            if (pnt > best_move_points):
                best_move_points = pnt
                best_move = c

    return best_move

        

In [8]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 2, 1)
play(board, 3, -1)
play(board, 2, 1)
play(board, 1, -1)
play(board, 4, 1)
play(board, 0, -1)
play(board, 4, 1)

print(board)

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


In [9]:
player = -1
stop = False
while (not four_in_a_row(board, -player) and not stop):
    print(board)
    print('')
    c = get_best_move(board, 1, 3)
    if (c == -1):
        print('no more moves')
        stop = True
    else:
        play(board, c, player)
    player = -player

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [-1 -1  0  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [-1 -1  1  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1 -1  0  0  0]
 [-1 -1  1  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1 -1  0  0  0]
 [-1 -1  1  0  0  0]
 [ 1  1  1  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1 -1  0  0  0]
 [-1 -1  1  0  0  0]
 [ 1  1  1 -1  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]

[[-1  1  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  1 -1  0  0  0]
 [-1 -1  1  0  0  0]
 [ 1  1  1 -1  0  0]
 